In [52]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import init
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from SSIM_PIL import compare_ssim
from PIL import Image

In [53]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [63]:
image2=plt.imread(r'C:\Users\rohan\Downloads\Miniproject (2)\Miniproject\data\image\fox.jpg')
image1=np.load(r"C:\Users\rohan\Downloads\Miniproject (2)\Miniproject\data\radon\fox.npy")

In [64]:
def nradon_transform(img):
    store = []
    for degree in range(360):
        rot= torchvision.transforms.functional.rotate(img.permute(2, 0, 1), degree)
        c = torch.exp(-0.01 * torch.cumsum(rot, 1))
        integral = torch.sum(rot * c, 1, keepdim=True)
        store.append(integral)
    r= torch.stack(store, 1).permute(2, 3, 1, 0)[0]
    return r

In [65]:
print(image1.shape)
x,y=np.meshgrid(np.linspace(0,1,512),np.linspace(0,1,512))
x=x.reshape((512*512),1)
y=y.reshape((512*512),1)
pixels=np.hstack((x,y))
pixels=torch.Tensor(pixels)
transforms=torchvision.transforms.ToTensor()
image1=transforms(image1)
print(pixels.numpy().shape)

(512, 180, 3)
(262144, 2)


In [66]:
def psnr(est,gt):
    psnr =-20*np.log10(np.linalg.norm((gt-est).reshape(-1))/(np.sqrt(gt.size)*np.max(gt)))
    return psnr

In [67]:
class Implicit(nn.Module):
    def __init__(self):
        super(Implicit, self).__init__()
        self.linear1=nn.Linear(2,128)
        self.relu1=nn.ReLU()
        self.linear2=nn.Linear(128,256)
        self.linear3=nn.Linear(256,512)
        self.linear4=nn.Linear(512,512)
        self.linear5=nn.Linear(512,256)
        self.linear6=nn.Linear(256,128)
        self.linear7=nn.Linear(128,64)
        self.linear8=nn.Linear(64,32)
        self.linear9=nn.Linear(32,3)
        self.a=nn.Sigmoid()
    def forward(self,x):
        x=self.linear1(x)
        x=self.relu1(x)
        x=self.linear2(x)
        x=self.relu1(x)
        x=self.linear3(x)
        x=self.relu1(x)
        x=self.linear4(x)
        x=self.relu1(x)
        x=self.linear5(x)
        x=self.relu1(x)
        x=self.linear6(x)
        x=self.relu1(x)
        x=self.linear7(x)
        x=self.relu1(x)
        x=self.linear8(x)
        x=self.relu1(x)
        x=self.linear9(x)
        x=self.a(x)
        return x

In [68]:
implicit=Implicit()
implicit.to(device)
criterion = nn.MSELoss() 
optimizer = torch.optim.Adam(implicit.parameters(),lr = 0.001)
n_epochs=10000
x=pixels
train_loss = 0
epochtrack=[]
psnrtrack=[]
simtrack=[]
trainlosstrack=[]
for epoch in range(n_epochs):
    optimizer.zero_grad()
    out=implicit(x.to(device))
    out=out.reshape(3,512,512)
    newoutput=nradon_transform(out.to(device),180)
    newoutput=newoutput.reshape(512,180,3)
    image1=image1.reshape(512,180,3)
    loss=criterion(newoutput,image1)
    loss.backward()
    optimizer.step()
    trainlosstrack.append(loss.item())
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, loss.item()))
    newoutput=newoutput.reshape(512,180,3)
    image1=image1.reshape(512,180,3)
    image2.reshape(512,512,3)
    psnrvalue=psnr(out.cpu().detach().numpy().reshape(512,512,3),image2)
    psnrtrack.append(psnrvalue)
    value = compare_ssim(Image.fromarray((out.cpu().detach().numpy().reshape(512,512,3)*255).astype(np.uint8)),Image.fromarray((image2*255).astype(np.uint8)))
    simtrack.append(value)

Epoch: 0 	Training Loss: 20712.425781
No module named 'pyopencl'
Epoch: 1 	Training Loss: 20697.410156
No module named 'pyopencl'
Epoch: 2 	Training Loss: 20682.394531
No module named 'pyopencl'
Epoch: 3 	Training Loss: 20665.660156
No module named 'pyopencl'
Epoch: 4 	Training Loss: 20642.875000
No module named 'pyopencl'
Epoch: 5 	Training Loss: 20613.515625
No module named 'pyopencl'
Epoch: 6 	Training Loss: 20577.650391
No module named 'pyopencl'
Epoch: 7 	Training Loss: 20521.330078
No module named 'pyopencl'
Epoch: 8 	Training Loss: 20428.253906
No module named 'pyopencl'
Epoch: 9 	Training Loss: 20282.945312
No module named 'pyopencl'
Epoch: 10 	Training Loss: 20084.054688
No module named 'pyopencl'
Epoch: 11 	Training Loss: 19853.371094
No module named 'pyopencl'
Epoch: 12 	Training Loss: 19615.300781
No module named 'pyopencl'
Epoch: 13 	Training Loss: 19411.115234
No module named 'pyopencl'
Epoch: 14 	Training Loss: 19265.982422
No module named 'pyopencl'
Epoch: 15 	Training 

KeyboardInterrupt: 

In [ ]:
print(trainlosstrack)
plt.plot(psnrtrack)
plt.title('PSNR vs Epochs')
plt.show()
plt.plot(simtrack)
plt.title('SSIM vs Epochs')
plt.show()
out=implicit(pixels)
plt.imshow(out.reshape(512,180,1).detach().numpy())
plt.show()
plt.imshow(image1)
plt.show()
image1=image1.reshape(512*180*1)
out=out.reshape(512*180*1)
print(out.shape)
print(image1.shape)